In [1]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import os
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle

In [2]:
#to switch target line
target = "Ständige Wohnbevölkerung Total"
#target ='Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'

# Prepare data for model
Read Data from excel

In [3]:
def read_excel():
    column_names_to_load = [
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien',
        target]

    # Load all data from excel
    path = 'Data/Preparation/Kennzahlen_aller_Gemeinden/Alle_Daten.xlsx'
    data = pd.read_excel(path, header=[0,1], index_col=[0,1])

    # Extract wanted columns
    column_mask = data.columns.isin(column_names_to_load, level=1)
    data = data.loc[:, column_mask]

    # Drop top level column names
    data = data.droplevel(0,axis=1)

    # Reset index
    data = data.reset_index()

    
    return data

Define Train / Verification / Test split

In [4]:
def train_test_split(data, test_length):
    start_year = 1991
    amount_of_years = 30

    # Define length of splits
    train_length = amount_of_years - test_length

    # Define years where train and validation split ends
    train_data_end_year = start_year + train_length

    # Apply splits
    train = data[data['Jahr'] <= train_data_end_year]
    test = data[data['Jahr'] > train_data_end_year]
    
    return train, test

Define X and Y Split

In [5]:
def x_y_split(data):
    y_column_name = target

    x_column_names = [
        'Jahr',
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien'
        ]

    # Execute the Y split
    y = data[y_column_name]
    y.name = "y"

    x = data[x_column_names]

    assert len(x) == len(y), 'X and Y split need to have the same length'

    return x, y

Set dummy variables and scale the data

In [6]:
def set_dummy_variables(x):
    # Set dummie variables for the column Gemeindetypologien and Gemeindename in the X split
    return pd.get_dummies(x, columns=['Gemeindetypologien', 'Gemeindename'])

def scale(x, scaler):
    # Store the column names that it can be reapplied after scaling
    x_columns = x.columns

    # Scale the data with the StandardScaler
    if not scaler:
        scaler = StandardScaler()
        x = scaler.fit_transform(x)
    else:
        x = scaler.transform(x)
    
    # Create pandas dataframes from ndArrays and reapply columnnames
    x = pd.DataFrame(x, columns=x_columns)

    return x, scaler

Extract data which is used for visualisation

In [7]:
def get_jahre(x):
    # All years from the x set
    return list(x['Jahr'].unique())

def get_gemeinde_and_gemeindetypologien(x):
    # All Gemeinde dummy variable column names
    gemeinden = x.columns.tolist()[11:]

    # All Gemeindetypologien dummy variable column names
    gemeinden_typologien = x.columns.tolist()[6:10]

    return gemeinden, gemeinden_typologien

# Model Evaluation

In [8]:
def clean_output(name): 
    directory = f'./{name}'

    prepare_folder(directory)
    prepare_folder(f'{directory}/Gemeinden')

    return directory

def prepare_folder(folder):
    # create folder if it doesn't exist otherwise clear the entire folder
    if not os.path.exists(folder):
        os.makedirs(folder)
    else: 
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
            except Exception as e:
                print(f"Error deleting file: {file_path} - {e}")

## Metrics

In [21]:
def evaluate(folder, y, y_pred, coeff):
    # calculate the R2 and MAPE and write it to a file
    r2 = r2_score(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)
    with open(f'{folder}/metric.txt', 'w') as f:
        f.write(f"R2: {r2} \n")
        f.write(f"MAPE: {mape}")

    # write the coefficients from the model to a file
    with open(f'{folder}/coeff.txt', 'w') as f:
        for key, value in coeff.items():
            f.write(f'{key}: {value}\n')

In [10]:
def get_coeff(model):
    # get the coefficients from the diffrent models
    type_switch = {
        SVR: lambda x: x.dual_coef_,
    }

    return type_switch[type(model)](model).tolist()

## Visualtions

## Plot for a single gemeinde

In [11]:
def plot_gemeinde(folder, gemeinden, jahre_train, jahre_test, x_list, y_list, y_pred_list):    
    # Concat the train and validation set for the X data set
    x = pd.concat(x_list, axis=0).reset_index()

    # Concat the train and validation set for the Y data set
    y = pd.concat(y_list, axis=0).reset_index()
    
    # Concat the predicted values from the train and validation set
    y_pred = pd.concat(y_pred_list, axis=0).reset_index()

    # Concat the X, Y, and the predicted values
    visualisation_all_gemeinde_data = pd.concat([pd.DataFrame(x), y, y_pred], axis=1)

    # Loop over all gemeinde
    for gemeinde in gemeinden:

        # Extract the data for one gemeinde. Get the rows where the gemeinde dummy variable is positive
        visualisation_gemeinde_data = visualisation_all_gemeinde_data[visualisation_all_gemeinde_data[gemeinde] > 0]

        # Extract the expected and the predicted value
        gemeinde_data_y = visualisation_gemeinde_data['y']
        gemeinde_pred = visualisation_gemeinde_data['pred']

        # Plot the data
        plt.figure()

        # Set title on plot
        plt.title(gemeinde)
        
        # plot the expected data as dots
        plt.scatter(jahre_train, gemeinde_data_y[:len(jahre_train)], label='Train')
        plt.scatter(jahre_test, gemeinde_data_y[len(jahre_train):len(jahre_train) + len(jahre_test)], color='lightgreen', label='Test')

        # plot the prediction as line
        plt.plot(jahre_train + jahre_test, gemeinde_pred, color="black", label='Prediction')

        # Show legend
        plt.legend()

        # activate this for fixed y-axis
        # plt.ylim([50, 2500])

        #plt.show()

        # Save the image and close the plot that it doesn't display in the jupyter notebook
        plt.savefig(f'{folder}/Gemeinden/{gemeinde}.png')
        plt.close()
      


## Plot to compare Gemeindetypologien

Plot standard deviation and absolut percentage error

In [12]:
def plot_std(folder, dataframe, gemeinden_typologien):
    # create a figure with subplots
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,8))
    fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)


    # iterate over the columns and plot each one in a subplot
    for ax, gemeinden_typologie in zip(axes.flat, gemeinden_typologien):
        # extract the column 
        subset = dataframe[dataframe[gemeinden_typologie] > 0][['Delta_Percent']]
        column_data = subset['Delta_Percent']

        # calculate the mean and standard deviation
        mean = column_data.mean()
        std = column_data.std()

        # plot the density plot with a shaded area for the standard deviation and a line for the mean
        ax = sns.kdeplot(column_data, fill=False, ax=ax, color='red', linewidth=1, label='absolut percentage error', bw_adjust=0.3)
        ax.axvline(mean, color='black', linestyle='dotted', linewidth=1, label = 'mean absolute percentage error: ' + str(round(mean, 1)))
        ax.fill_betweenx([0, ax.get_ylim()[1]], mean-std, mean+std, alpha=0.3, color='blue', label="standard deviation: " + str(round(std, 1)) + " %")

        #labels
        ax.set_xlabel('absolut percentage error')
        ax.set_ylabel('density')
        ax.set_title(gemeinden_typologie, fontsize=10)
        ax.legend()
        ax.grid(False)


    # set same scale for all subplots
    for ax in axes.flat:
        ax.set_xlim([-5, 25])
        ax.set_ylim([0, 0.28])
        ax.tick_params(axis='both', labelsize=8)

    # adjust the layout and show the figure
    fig.tight_layout()
    #plt.show()

    #Save the image and close the plot that it doesn't display in the jupyter notebook
    plt.savefig(f'{folder}/standard_deviation.png')
    plt.close()

Plot density of percentage error

In [13]:
def plot_density(folder, visualisation_all_gemeindetypologien_data):
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

    # Create density plot
    sns.set_style('whitegrid')
    sns.kdeplot(visualisation_all_gemeindetypologien_data['Delta_Percent'], fill=True, color='red', bw_adjust=0.5)
    
    # Plot mean and median
    mean = np.mean(visualisation_all_gemeindetypologien_data['Delta_Percent'])
    plt.axvline(x=mean, color='black', linestyle='dotted', linewidth=1.3, label='mean absolut percentage error: ' + str(round(mean, 1)))
    median = np.median(visualisation_all_gemeindetypologien_data['Delta_Percent'])
    plt.axvline(x=median, color='blue', linestyle='dotted', linewidth=1.3, label='median absolut percentage error: ' + str(round(median, 1)))
    
    # Calculate 15% and 85% quantiles
    cutoff15 = np.percentile(visualisation_all_gemeindetypologien_data['Delta_Percent'], q=85)
    cutoff85 = np.percentile(visualisation_all_gemeindetypologien_data['Delta_Percent'], q=15)

    # Plot quantiles
    plt.axvline(x=cutoff15, color='red', linestyle='dotted', linewidth=1.3, label='15% quantile: '+ str(round(cutoff85, 1)))
    plt.axvline(x=cutoff85, color='red', linestyle='dotted', linewidth=1.3, label='85% quantile: ' + str(round(cutoff15, 1)))

    # Add labels and title
    plt.xlabel('absolut percentage error', labelpad=10)
    plt.ylabel('density', labelpad=10)
    plt.title('distribution - absolut percentage error', fontsize=10)
    plt.tick_params(axis='both', labelsize=8)
    
    # Show legend
    plt.legend()
    plt.grid(False)

    #plt.show()

    # Save the image
    plt.savefig(f'{folder}/density.png')
    plt.close()
    

Violin plot of percentage error for each gemeindetypologie

In [14]:
def plot_violin_plot(folder, gemeinden_typologien, visualisation_all_gemeindetypologien_data):
    plt.figure(figsize=(6, 4))

    # Extract the data for each Gemeindetypologie
    gemeinden_typologien_violin_party = []

    # Loop over all Gemeindetypologien
    for gemeinden_typologie in gemeinden_typologien: 
        gemeinden_typologien_violin_party.append(
            # Extract the data for one Gemeindetypologie. Get the rows where the Gemeindetypologie dummy variable is positive
            visualisation_all_gemeindetypologien_data[visualisation_all_gemeindetypologien_data[gemeinden_typologie] > 0]['Delta_Percent']
        )
    

    # Create violin plot
    violin_parts = plt.violinplot(gemeinden_typologien_violin_party, showmeans=False, showmedians=True)

    # xtick labels
    typologien_list = [''] + [gemeinden_typologie[19:] for gemeinden_typologie in gemeinden_typologien] + ['']

    # Set title
    plt.title("Absolute Percentage error of predictions", fontsize=12, pad=20)

    # Set lables
    plt.xlabel('Gemeindetypologien', fontsize=12, labelpad=10)
    plt.ylabel('Absolute Percentage Error', fontsize=12, labelpad=10)
    plt.xticks(list(range(len(typologien_list))), typologien_list, fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True)

    # Set color
    for pc in violin_parts['bodies']:
        pc.set_facecolor('red')
        pc.set_color('red')
        pc.set_edgecolor('black')

    for partname in ('cbars','cmins','cmaxes','cmedians'):
        vp = violin_parts[partname]
        vp.set_edgecolor('red')
        vp.set_linewidth(1)
    
    #plt.show()
    
    # Save the image
    plt.savefig(f'{folder}/topologien.png')
    plt.close()
    


Function to plot all of them and prepare the data

In [15]:
def plot_gemeinde_typologie(folder, gemeinden_typologien, x, y, y_pred):
    # Concat the X, Y, and the predicted values
    visualisation_all_gemeindetypologien_data = pd.concat([x.reset_index(), y.reset_index(), y_pred.reset_index()], axis=1)
    

    # Calculate delta for each prediction
    visualisation_all_gemeindetypologien_data['Delta'] = visualisation_all_gemeindetypologien_data['pred'] - visualisation_all_gemeindetypologien_data['y']   

    # Calculate the absolute percentage error for each prediction
    visualisation_all_gemeindetypologien_data['Delta_Percent'] = 100
    visualisation_all_gemeindetypologien_data['Delta_Percent'] = (abs(visualisation_all_gemeindetypologien_data['Delta']) / visualisation_all_gemeindetypologien_data['pred'] ) * 100
    #return visualisation_all_gemeindetypologien_data
    plot_std(folder, visualisation_all_gemeindetypologien_data, gemeinden_typologien)
    plot_violin_plot(folder, gemeinden_typologien, visualisation_all_gemeindetypologien_data)
    plot_density(folder, visualisation_all_gemeindetypologien_data)


# Define size of train & validation split
Size of test set is fixed to 8 points
SVR POLY3 C=80 Gamma=0.015

In [16]:
TRAIN_LENGTH = 22
TEST_LENGTH = 8

model_name = f'T{TRAIN_LENGTH} V{TEST_LENGTH} SVR POLY3 C=80 Gamma=0.015'
model = SVR(kernel='poly', degree=3, C=30, gamma=0.1)

# Load data if not already loaded
data = read_excel()

# Split
train, test = train_test_split(data, TEST_LENGTH)

x_train, y_train = x_y_split(train)
x_test, y_test = x_y_split(test)

# Set dummy variables
x_train = set_dummy_variables(x_train)
x_test = set_dummy_variables(x_test)

# Extract data for visualisations before scaling
jahre_train = get_jahre(x_train)
jahre_test = get_jahre(x_test)
gemeinden, gemeinden_typologien = get_gemeinde_and_gemeindetypologien(x_test)

# Scale the data
x_train, scaler = scale(x_train, None)
x_test, scaler = scale(x_test, scaler)

# Train the model
SVR POLY3 C=80 Gamma=0.015

In [17]:
print("Trying model:", model_name)

# Prepare folder for output
directory = clean_output(model_name)

# Train Model
model.fit(x_train, y_train)

# or load already stored weights if desired
# with open(f'{directory}/model_weights.pkl', 'rb') as f:
#     model = pickle.load(f)

# Save the trained model weights
with open(f'{directory}/model_weights.pkl', 'wb') as f:
    pickle.dump(model, f)


# Prediction
y_pred_train = pd.Series(model.predict(x_train), name = 'pred')
y_pred_test = pd.Series(model.predict(x_test), name = 'pred')
        

Trying model: T22 V8 SVR POLY3 C=80 Gamma=0.015


# Evaluate the model

In [23]:
coeff = get_coeff(model)
# dict comprehension of coeff and x_train.columns
coeff_columns = {k: v for k, v in zip(x_train.columns, *coeff)}
evaluate(directory, y_test, y_pred_test, coeff_columns)
print(coeff_columns)


{'Jahr': -30.0, 'Siedlungsfläche_in_%': 30.0, 'Landwirtschafts-fläche_in_%': -30.0, 'Betriebe_total': 30.0, 'Wohnungen - Total': 30.0, 'Anzahl_Privathaushalte': 30.0, 'Gemeindetypologien_Aggloguertel': -30.0, 'Gemeindetypologien_Agglokern': -17.118545964734313, 'Gemeindetypologien_Kern': -30.0, 'Gemeindetypologien_Land': -30.0, 'Gemeindename_Adligenswil': 30.0, 'Gemeindename_Alberswil': 30.0, 'Gemeindename_Altbüron': 30.0, 'Gemeindename_Altishofen': -30.0, 'Gemeindename_Ballwil': -30.0, 'Gemeindename_Beromünster': 30.0, 'Gemeindename_Buchrain': 14.69568459312683, 'Gemeindename_Buttisholz': -30.0, 'Gemeindename_Büron': -30.0, 'Gemeindename_Dagmersellen': -30.0, 'Gemeindename_Dierikon': 30.0, 'Gemeindename_Doppleschwand': -4.301197380476823, 'Gemeindename_Ebikon': 30.0, 'Gemeindename_Egolzwil': -30.0, 'Gemeindename_Eich': -30.0, 'Gemeindename_Emmen': -30.0, 'Gemeindename_Entlebuch': 16.880545629521578, 'Gemeindename_Ermensee': 30.0, 'Gemeindename_Escholzmatt-Marbach': 30.0, 'Gemeindename

Plot the gemeinden

In [19]:
plot_gemeinde(directory, 
    gemeinden, 
    jahre_train, 
    jahre_test,
    [x_train, x_test], 
    [y_train, y_test], 
    [y_pred_train, y_pred_test]
    )

Plot evaluations 

In [20]:
plot_gemeinde_typologie(directory, 
                        gemeinden_typologien, 
                        x_test, 
                        y_test,
                        y_pred_test
                            )

- aussortieren gemeinden, schlechte performance?
--> wie umgang mit gemeinden schlecht predicted